In [ ]:
import numpy as np                	
import matplotlib.pyplot as plt   	
import pandas as pd               	
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from tensorflow.keras import regularizers
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score


In [ ]:
dataset = pd.read_csv('features.csv')
dataset.info()
dataset = dataset[['id', 'organization_num', 'productivity', 'title_length', 'H_index', 'n_authors', 'journal_publications', 'prev_citation_sum', 'faculty_impact', 'max_H_index', 'references_ave_age', 'self_cite_percent', 'journal_percent', 'conference_percent', 'patent_percent', 'book_percent', 'paper_len', 'avg_cite', 'n_papers', 'n_references', 'doc_type', 'n_citation']]
dataset = dataset.drop(['journal_publications', 'prev_citation_sum', 'avg_cite'], axis=1)
dataset['doc_type'] = dataset['doc_type'].fillna('others')

X = dataset.iloc[:, 1:18].values
y = dataset.iloc[:, 18].values

imputer = SimpleImputer(missing_values = np.nan, strategy = 'constant', fill_value=-0.1)
imputer.fit(X[:, 5:16])
X[:, 5:16] = imputer.transform(X[:, 5:16])
x = X[:, 16]

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [16])] , remainder='passthrough')
X = np.array(ct.fit_transform(X))

df = pd.DataFrame.from_records(X)
df = df.drop(4, axis = 1)
X = df.values.tolist()

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.20, random_state = 0)


In [ ]:
predictor  = Sequential()
predictor.add(Dense(units =10, kernel_regularizer = regularizers.l2(0.003)))

predictor.add(LeakyReLU(alpha = 0.0015))
predictor.add(Dense(units = 30,kernel_regularizer = regularizers.l2(0.003)))
predictor.add(LeakyReLU(alpha = 0.0015))
predictor.add(Dense(units = 1,kernel_regularizer = regularizers.l2(0.003), activation='relu'))
predictor.compile(optimizer = 'adam' , loss = 'mean_squared_error' , metrics = ['accuracy'])

print(type(X_train))
y_train = np.array(y_train)
X_train = np.array(X_train)
predictor.fit(X_train, y_train, batch_size = 64, epochs = 100)

y_predict = predictor.predict(X_test)
print(type(y_predict))

In [ ]:
X_test = np.array(X_test)
accuracy = predictor.evaluate(X_test, y_test)
print(accuracy)

X_train = np.array(X_train)
accuracy = predictor.evaluate(X_train, y_train)
print(accuracy)

y_predict = np.round(y_predict)

y_diff = y_test - y_predict

mean_squared_error(y_test,y_predict)
print(mean_squared_error(y_test,y_predict))

mean_absolute_error(y_test,y_predict)
print(mean_absolute_error(y_test,y_predict))

r2_score(y_test,y_predict)
print(r2_score(y_test,y_predict))

accuracy_score(y_test,y_predict)
print(accuracy_score(y_test,y_predict))

accuracy1 = predictor.evaluate(X_test, y_test)
accuracy2 = predictor.evaluate(X_train, y_train)
MSE = mean_squared_error(y_test,y_predict)
MAE = mean_absolute_error(y_test,y_predict)